# Explainable Medical Diagnosis AI - Demo Notebook
This notebook demonstrates how to load data, train a basic CNN model on medical images, and apply Grad-CAM for visual explanations.


## Step 1: Load Data

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Sample synthetic data loading setup
image_dir = 'data/sample_images/'
img_size = 128

# Load and preprocess images
def load_images(directory, size=(img_size, img_size)):
    images = []
    labels = []
    for fname in os.listdir(directory):
        if fname.endswith('.jpg') or fname.endswith('.png'):
            img = cv2.imread(os.path.join(directory, fname))
            img = cv2.resize(img, size)
            images.append(img)
            labels.append(1 if 'positive' in fname else 0)
    return np.array(images), np.array(labels)

X, y = load_images(image_dir)
X = X / 255.0
print(f"Loaded {len(X)} images with shape {X[0].shape}")


## Step 2: Train CNN Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# Simple CNN model
model = Sequential([
    Conv2D(16, (3,3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D(2,2),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train (limited epochs for demo)
history = model.fit(X, y, epochs=3, validation_split=0.2)


## Step 3: Grad-CAM for Explainability

In [ ]:
import tensorflow as tf

# Grad-CAM function
def grad_cam_heatmap(model, image, layer_name='conv2d_1'):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(np.expand_dims(image, axis=0))
        loss = predictions[:, 0]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = np.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# Select and visualize one image
img = X[0]
heatmap = grad_cam_heatmap(model, img)

# Superimpose heatmap on original image
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
heatmap_img = np.uint8(255 * heatmap)
heatmap_img = cv2.applyColorMap(heatmap_img, cv2.COLORMAP_JET)
superimposed = cv2.addWeighted(np.uint8(img * 255), 0.6, heatmap_img, 0.4, 0)

# Display result
plt.figure(figsize=(10, 4))
plt.subplot(1, 3, 1)
plt.title("Original Image")
plt.imshow(img)
plt.axis('off')

plt.subplot(1, 3, 2)
plt.title("Grad-CAM Heatmap")
plt.imshow(heatmap, cmap='jet')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.title("Superimposed")
plt.imshow(superimposed)
plt.axis('off')
plt.show()
